## 1. 
## 2. 

In [38]:
import json

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
# from tensordict import TensorDict

pd.set_option('display.max_info_columns', 500)

In [ ]:
# !pip install torch

In [39]:
with open(
    "/Users/leegary/Downloads/mimic_iv_multilabel__json_files__20240302/train_set__chexpert.json"
) as f:
    data = json.load(f)
    np.asarray(data)

How Many 

In [43]:
data = pd.DataFrame(data)
data['acuity'].nunique()

4

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9630 entries, 0 to 9629
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   patient_id                            9630 non-null   float64
 1   visit_id                              9630 non-null   int64  
 2   study_id                              9630 non-null   float64
 3   temperature                           9630 non-null   float64
 4   heartrate                             9630 non-null   float64
 5   resprate                              9630 non-null   float64
 6   o2sat                                 9630 non-null   float64
 7   sbp                                   9630 non-null   float64
 8   dbp                                   9630 non-null   float64
 9   pain                                  9630 non-null   int64  
 10  acuity                                9630 non-null   float64
 11  pathologies_number    

In [44]:
print(data['patient_id'].nunique())

9630


1. Import scikit-learn... Download most recent data from Esteban

2. Combine test, train, val  (identify them with 2 cols [test/train/val] split) - 16,907 (train 11)


    Full Set
    ```
    0      test         2817   16.661738
    1     train        11272   66.670610
    2  validate         2818   16.667652
    ```
    Sample Set
    ```
    data_type  is_sample  count
    1      test       True    282
    3     train       True   1127
    5  validate       True    282
    ```

3.  Combine sample sets patient IDs (test, train, val)
    - Change acuity to int64 and remove all unneeded data
    - Keep columns 0, 3 - 10, 20-25 and remove everything else except the target variables
    - Index the resulting file by patientID 
    - Split all to x features and y labels

4. create a pipeline in scikit-learn to predict a multi classification outcome target variable with softmax probabilities 
    - Confirm whether any values are missing
    - Scale features 
    - Decide datatypes (Ordinal, Ratio)
    - Use XGBoost for robustness (this will be for early fusion)

5. Hyperparameter tuning using validation set
    - Use log loss 
    -  Gridsearch 

6. Prediction testing and scoring metrics (precision, recall, f1 score, and auc)

In [ ]:
input_cols = [
    "patient_id",
    "visit_id",
    "study_id",
    "temperature",
    "heartrate",
    "resprate",
    "o2sat",
    "sbp",
    "dbp",
]
output_cols = [
    "atelectasis",
    "cardiomegaly",
    "edema",
    "lung_opacity",
    "pleural_effusion",
    "pneumonia",
]
input_df = pd.DataFrame(
    data=data,
    columns=input_cols,
)
output_df = pd.DataFrame(data=data, columns=output_cols)
input_df = input_df.astype(
    {
        "patient_id": int,
        "study_id": int,
    }
)
output_df = output_df.astype(
    {
        "atelectasis": int,
        "cardiomegaly": int,
        "edema": int,
        "lung_opacity": int,
        "pleural_effusion": int,
        "pneumonia": int,
    }
)

In [ ]:
input_df.head()

In [ ]:
def scale_min_max(df, col_name):
    xmin = df[col_name].min()
    xmax = df[col_name].max()
    df[col_name] = (df[col_name] - xmin) / (xmax - xmin)
    return df

In [ ]:
input_df = scale_min_max(input_df, "temperature")
input_df = scale_min_max(input_df, "heartrate")
input_df = scale_min_max(input_df, "resprate")
input_df = scale_min_max(input_df, "sbp")
input_df = scale_min_max(input_df, "dbp")
input_df["o2sat"] = input_df["o2sat"] * 0.01

In [ ]:
input_tensor = torch.tensor(input_df.values)

In [ ]:
output_tensor = torch.tensor(output_df.values)

In [ ]:
test = TensorDataset(input_tensor, output_tensor)

In [ ]:
test[0][1]

In [ ]:
output_df.head(1)

In [ ]:
test[0][0]

In [ ]:
input_df.head(1)

In [ ]:
def get_tabular_df(path):
    """
    Generates the tabular dataframes
    """
    with open(path) as f:
        data = json.load(f)
    index_cols = ["patient_id", "visit_id", "study_id"]
    input_cols = [
        "patient_id",
        "visit_id",
        "study_id",
        "temperature",
        "heartrate",
        "resprate",
        "o2sat",
        "sbp",
        "dbp",
    ]
    output_cols = [
        "atelectasis",
        "cardiomegaly",
        "edema",
        "lung_opacity",
        "pleural_effusion",
        "pneumonia",
    ]
    input_df = pd.DataFrame(
        data=data,
        columns=input_cols,
    )
    output_df = pd.DataFrame(data=data, columns=output_cols)
    input_df = input_df.astype(
        {
            "patient_id": int,
            "study_id": int,
        }
    )
    output_df = output_df.astype(
        {
            "atelectasis": int,
            "cardiomegaly": int,
            "edema": int,
            "lung_opacity": int,
            "pleural_effusion": int,
            "pneumonia": int,
        }
    )
    return [input_df, output_df]

In [ ]:
def generate_tabular(path, name, all_inputs):
    """
    Generates the tabular tensor files for loading.
    """
    with open(path) as f:
        data = json.load(f)
    input_cols = [
        "patient_id",
        "visit_id",
        "study_id",
        "temperature",
        "heartrate",
        "resprate",
        "o2sat",
        "sbp",
        "dbp",
        "atelectasis",
        "cardiomegaly",
        "edema",
        "lung_opacity",
        "pleural_effusion",
        "pneumonia",
    ]
    input_df = pd.DataFrame(
        data=data,
        columns=input_cols,
    )
    input_df = input_df.astype(
        {
            "patient_id": int,
            "study_id": int,
            "atelectasis": int,
            "cardiomegaly": int,
            "edema": int,
            "lung_opacity": int,
            "pleural_effusion": int,
            "pneumonia": int,
        }
    )

    def scale_min_max(df, col_name):
        xmin = all_inputs[col_name].min()
        xmax = all_inputs[col_name].max()
        df[col_name] = (df[col_name] - xmin) / (xmax - xmin)
        return df

    # normalize
    input_df = scale_min_max(input_df, "temperature")
    input_df = scale_min_max(input_df, "heartrate")
    input_df = scale_min_max(input_df, "resprate")
    input_df = scale_min_max(input_df, "sbp")
    input_df = scale_min_max(input_df, "dbp")
    input_df["o2sat"] = input_df["o2sat"] * 0.01

    input_df.to_csv("../data/s3/" + name, index=False)
    print("generated " + name)

In [ ]:
root = "/Users/leegary/Downloads/mimic_iv_multilabel__json_files__20240302/"
paths = [
    ["tabular_test.csv", "test_set__chexpert.json"],
    ["tabular_train.csv", "train_set__chexpert.json"],
    ["tabular_valid.csv", "validation_set__chexpert.json"],
]
inputs = []
outputs = []
for [name, path] in paths:
    [input_df, output_df] = get_tabular_df(root + path)
    inputs.append(input_df)

all_inputs = pd.concat(inputs, axis=0)

for [name, path] in paths:
    generate_tabular(root + path, name, all_inputs)

In [ ]:
np.array(input_df.columns)

In [ ]:
np.array(output_df.columns)

In [ ]:
np.loadtxt("../data/s3/tensors/tabular_label_names.txt", delimiter=",", dtype="str")

In [ ]:
from types import SimpleNamespace

import mimic

In [ ]:
config_test = '{"data_mode": "tabular_numpy_train", "batch_size": 32}'
x = json.loads(config_test, object_hook=lambda d: SimpleNamespace(**d))
dataloaders = mimic.MimicDataLoader(x)

In [ ]:
next(iter(dataloaders.train_loader))